# **DNF F1 analasys project** 

## Part 1 - Loading and maipulating data

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

In [77]:
#Importing data to dataframe
data = pd.read_csv('data.csv')
print(data.head(1).to_string() ,"\n")
print("Shape of daraframe: ", data.shape)

   resultId  raceId  year  round  grid  positionOrder  points  laps milliseconds fastestLap rank fastestLapTime fastestLapSpeed  driverRef    surname forename         dob nationality_x constructorRef         name nationality_y   circuitRef  circuitId   name_y  location  country      lat      lng  alt        date  target_finish
0      2460     136  2002     13    11              4     3.0  77.0          NaN         \N   \N             \N              \N  raikkonen  Räikkönen     Kimi  1979-10-17       Finnish        mclaren  Hungaroring       British  hungaroring         11  McLaren  Budapest  Hungary  47.5789  19.2486  264  2002-08-18              1 

Shape of daraframe:  (10000, 31)


As we can see, we have 10 000 rows/cars and 31 columns/features. A lot of these columns have Nan or empty values, we therefore have to remove these to get the desired features that can be used in our models.

In [78]:
#Finding number of null values per colum
nullValues = data.isnull().sum()
print(nullValues, "\n")
print(len(nullValues))



resultId              0
raceId                0
year                  0
round                 0
grid                  0
positionOrder         0
points              971
laps                978
milliseconds       1018
fastestLap            0
rank                  0
fastestLapTime        0
fastestLapSpeed     953
driverRef             0
surname               0
forename              0
dob                   0
nationality_x         0
constructorRef        0
name                  0
nationality_y         0
circuitRef            0
circuitId             0
name_y                0
location              0
country               0
lat                   0
lng                   0
alt                   0
date                  0
target_finish         0
dtype: int64 

31


We can see there are 4 columns that have 9-10% NAN values, adn the rest have 0 NAN columns. We therefore decide to remove these columns. 

In [79]:
data = data.dropna()
print(data.isnull().sum())
len(data.columns)


resultId           0
raceId             0
year               0
round              0
grid               0
positionOrder      0
points             0
laps               0
milliseconds       0
fastestLap         0
rank               0
fastestLapTime     0
fastestLapSpeed    0
driverRef          0
surname            0
forename           0
dob                0
nationality_x      0
constructorRef     0
name               0
nationality_y      0
circuitRef         0
circuitId          0
name_y             0
location           0
country            0
lat                0
lng                0
alt                0
date               0
target_finish      0
dtype: int64


31

Now our dataset consists of 31 columns where no columns have NAN values

Some columns have leakage features, meaning the column gives away the answer of what we are trying to predict. These need to be removed as they will make the predictions too easy.

We also need to decide which of the non-numerical columns to discard even if they are not leakage features. Simply because they have too diverse (sparse) values relative to the amount of data. There is not enoigh data to see a pattern with this many values

As an example, the first name of the driver will be too sparse for the model to learn what drivers are reckless, and which are not. 

In [81]:
data2.describe()
data2.corr(numeric_only=True)['target_finish'].sort_values(ascending=False)

target_finish    1.000000
points           0.628731
laps             0.334690
year             0.290462
resultId         0.153426
raceId           0.139581
round            0.132131
lng              0.093242
circuitId       -0.018930
lat             -0.019015
alt             -0.042671
grid            -0.334981
positionOrder   -0.627143
Name: target_finish, dtype: float64